In [18]:
#Installing dependencies
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 45.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 5.9 MB/s eta 0:00:00:00:0100:01


In [3]:
#Importing dependencies
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

### 1. Loading the data

In [4]:
#Reading input data
Train_data = pd.read_csv('train_v9rqX0R.csv')

In [5]:
#Displaying input data
display(Train_data.head())

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### 2. Exploratory Data Analysis

In [14]:
#Analyzing shape of input data
display(Train_data.shape)

(8523, 12)

In [15]:
#Analyzing datatypes of input data
display(Train_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


None

In [16]:
#Understanding data distribution
display(Train_data.describe())

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
#Profiling input data
Profiling = ProfileReport(Train_data, title = 'Sales')
Profiling.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 91.40it/s][A


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### 3. Model Training

In [7]:
#Importing dependencies
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error

In [8]:
#Identifying Target variable
y = Train_data.Item_Outlet_Sales
y

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [9]:
# Identifying columns of input data
Columns = Train_data.columns
Columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [10]:
#Feature Selection based upon correlation
features = ['Item_Identifier','Item_MRP','Item_Visibility','Outlet_Establishment_Year','Outlet_Identifier','Outlet_Location_Type','Outlet_Type']

In [11]:
#Sepearating out input features
X = Train_data[features]
X

,Item_Identifier,Item_MRP,Item_Visibility,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Type
0,FDA15,249.8092,0.016047,1999,OUT049,Tier 1,Supermarket Type1
1,DRC01,48.2692,0.019278,2009,OUT018,Tier 3,Supermarket Type2
2,FDN15,141.6180,0.016760,1999,OUT049,Tier 1,Supermarket Type1
3,FDX07,182.0950,0.000000,1998,OUT010,Tier 3,Grocery Store
4,NCD19,53.8614,0.000000,1987,OUT013,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...
8518,FDF22,214.5218,0.056783,1987,OUT013,Tier 3,Supermarket Type1
8519,FDS36,108.1570,0.046982,2002,OUT045,Tier 2,Supermarket Type1
8520,NCJ29,85.1224,0.035186,2004,OUT035,Tier 2,Supermarket Type1
8521,FDN46,103.1332,0.145221,2009,OUT018,Tier 3,Supermarket Type2


In [ ]:
# Separating categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns
for col in categorical_cols:
        X[col] = X[col].astype('category')

In [13]:
#Sepearating train & test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Training & selecting the best performing model

model = XGBRegressor(
    tree_method='hist', 
    enable_categorical=True, 
    eval_metric='rmse'
)

# Preparing list of hyperparamenter range for tuning
param_grid = {
    'n_estimators': [100,150,200],
    'max_depth': [2,3,4],
    'learning_rate': [0.02,0.05,0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8,1.0],
    'max_bin': [60, 70,80],
    'min_child_weight': [5,6,7],
    'lambda': [0.01,0.05],  
    'alpha': [0.05,0.1,] ,
    'gamma':[0,1]
                 }

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all cores
)

# Fitting model
grid_search.fit(X_train, y_train)

# Selecting best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 7776 candidates, totalling 23328 fits
Best Parameters: {'alpha': 0.05, 'colsample_bytree': 1.0, 'gamma': 0, 'lambda': 0.01, 'learning_rate': 0.05, 'max_bin': 70, 'max_depth': 2, 'min_child_weight': 6, 'n_estimators': 150, 'subsample': 0.8}


In [45]:
#Evaluating model performance on test data
y_pred = grid_search.predict(X_test)

# Assume y_test and y_pred are defined
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE: {rmse:.2f}")

array([1309.378  ,  818.09625,  873.09875, ...,  846.8138 ,  779.3139 ,
       1634.5829 ], dtype=float32)

### 5. Model performance evaluation

In [32]:
# Loading Hackathon test data
Test_data = pd.read_csv('test_AbJTz2l.csv')

In [ ]:
# Pre-processing test data
X_test_data = Test_data[features]
for col in categorical_cols:
    X_test_data[col] = X_test_data[col].astype('category')

In [34]:
#Evaluating model performance on Hackathon test data
y_pred_test = grid_search.predict(X_test_data)
Y_Pred = np.abs(y_pred_test)
# Saving the model prediction as csv file
np.savetxt('Predicted_values.csv', Y_Pred, delimiter=',', fmt='%d') 